In [1]:
import sympy as sp
import sympy.physics.mechanics as me

# creating constants
m,g,IUxx,IUyy,IUzz,IUxy,IUyz,IUzx,IMxx,IMyy,IMzz,IMxy, IMyz, IMzx,ILxx,ILyy,ILzz,ILxy, ILyz, ILzx,c,k = sp.symbols('m g IUxx IUyy IUzz IUxy IUyz IUzx IMxx IMyy IMzz IMxy IMyz IMzx ILxx ILyy ILzz ILxy ILyz ILzx c k')
x_cont1,y_cont1,z_cont1,x_cont2,y_cont2,z_cont2, k_contact, mx, my, mz, ax, ay, az, eps  = sp.symbols('x_cont1 y_cont1 z_cont1 x_cont2 y_cont2 z_cont2 k_contact mx my mz ax ay az eps')

# creating generalized coordinates, generalized forces and their derivatives
q1, q2, q3, q4, q5, q6, q7, q8, q9 = me.dynamicsymbols('q1 q2 q3 q4 q5 q6 q7 q8 q9')
q1d, q2d, q3d, q4d, q5d, q6d, q7d, q8d, q9d = me.dynamicsymbols('q1 q2 q3 q4 q5 q6 q7 q8 q9', 1)
u1, u2, u3, u4, u5, u6, u7, u8, u9 = me.dynamicsymbols('u1 u2 u3 u4 u5 u6 u7 u8 u9')
u1d, u2d, u3d, u4d, u5d, u6d, u7d, u8d, u9d = me.dynamicsymbols('u1 u2 u3 u4 u5 u6 u7 u8 u9', 1)
t = sp.Symbol('t')

# joint position from child and parent CoM (rigid1C = )
rigid0P = sp.symbols('rigid0Px:z')
rigid1P = sp.symbols('rigid1Px:z')
rigid1C = sp.symbols('rigid1Cx:z')
rigid2P = sp.symbols('rigid2Px:z')
rigid2C = sp.symbols('rigid2Cx:z')
rigid3C = sp.symbols('rigid3Cx:z')

# creating bodies
clavicle_com = me.Point('clavicle_com')
scapula_com = me.Point('scapula_com')
humerus_com = me.Point('humerus_com')
frame1 = me.ReferenceFrame('frame1')
frame2 = me.ReferenceFrame('frame2')
frame3 = me.ReferenceFrame('frame3')
clavicle_mass, scapula_mass, humerus_mass = sp.symbols('clavicle_mass scapula_mass humerus_mass')
thorax = me.Body('Th')
body_inertia1 = me.inertia(frame1, IUxx, IUyy, IUzz, IUxy, IUyz, IUzx)
body_inertia2 = me.inertia(frame2, IMxx, IMyy, IMzz, IMxy, IMyz, IMzx)
body_inertia3 = me.inertia(frame3, ILxx, ILyy, ILzz, ILxy, ILyz, ILzx)

clavicula = me.Body('Cl', clavicle_com, clavicle_mass, frame1, body_inertia1)
scapula = me.Body('Sc', scapula_com, scapula_mass, frame2, body_inertia2)
humerus = me.Body('Hu', humerus_com, humerus_mass, frame3, body_inertia3)

# creating joints
SC = me.SphericalJoint('SC', thorax, clavicula, 
                        child_point=rigid1C[0]*clavicula.x+rigid1C[1]*clavicula.y+rigid1C[2]*clavicula.z,
                        parent_point=rigid0P[0]*thorax.x+rigid0P[1]*thorax.y+rigid0P[2]*thorax.z,
                        coordinates = [q1, q2, q3],
                        speeds = [u1, u2, u3],rot_order='231', rot_type = 'body')
AC = me.SphericalJoint('AC', clavicula,  scapula,  
                        child_point=rigid2C[0]*scapula.x+rigid2C[1]*scapula.y+rigid2C[2]*scapula.z,
                        parent_point=rigid1P[0]*clavicula.x+rigid1P[1]*clavicula.y+rigid1P[2]*clavicula.z, 
                        coordinates = [q4, q5, q6],
                        speeds = [u4, u5, u6],rot_order='231', rot_type = 'body')
GH = me.SphericalJoint('GH', scapula,  humerus,  
                        child_point=rigid3C[0]*humerus.x+rigid3C[1]*humerus.y+rigid3C[2]*humerus.z,
                        parent_point=rigid2P[0]*scapula.x+rigid2P[1]*scapula.y+rigid2P[2]*scapula.z, 
                        coordinates = [q7, q8, q9],
                        speeds = [u7, u8, u9],rot_order='232', rot_type = 'body')

# applying gravitational forces in Y direction
clavicula.apply_force(-thorax.y*clavicula.mass*g)
scapula.apply_force(-thorax.y*scapula.mass*g)
humerus.apply_force(-thorax.y*humerus.mass*g)

# linear springs and dampings in joints
# angular velocities in base frame
w_clavicula = clavicula.ang_vel_in(thorax.frame)
w_scapula = scapula.ang_vel_in(clavicula.frame)
w_humerus = humerus.ang_vel_in(scapula.frame)
T1 = -c*w_clavicula.dot(frame1.x)*frame1.x
T2 = -c*w_clavicula.dot(frame1.y)*frame1.y
T3 = -c*w_clavicula.dot(frame1.z)*frame1.z
T4 = -c*w_scapula.dot(frame2.x)*frame2.x
T5 = -c*w_scapula.dot(frame2.y)*frame2.y
T6 = -c*w_scapula.dot(frame2.z)*frame2.z
T7 = -c*w_humerus.dot(frame3.x)*frame3.x
T8 = -c*w_humerus.dot(frame3.y)*frame3.y
T9 = -c*w_humerus.dot(frame3.z)*frame3.z
clavicula.apply_torque(T1+T2+T3,thorax)
scapula.apply_torque(T4+T5+T6,clavicula)
humerus.apply_torque(T7+T8+T9,scapula)

In [2]:
# Contact between scapula and thorax

# contact points 
contact_point1 = me.Point('CP1')
contact_point1.set_pos(scapula.masscenter,x_cont1*scapula.x+y_cont1*scapula.y  +z_cont1*scapula.z)
contact_point1.set_vel(scapula.frame,0) # point is fixed in scapula

contact_point2 = me.Point('CP2')
contact_point2.set_pos(scapula.masscenter,x_cont2*scapula.x+y_cont2*scapula.y  +z_cont2*scapula.z)
contact_point2.set_vel(scapula.frame,0) # point is fixed in scapula

## contact forces

# Distances between contact points and thorax frame
x_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.x)+x_cont1*scapula.x.dot(thorax.x)+y_cont1*scapula.y.dot(thorax.x)+z_cont1*scapula.z.dot(thorax.x)
y_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.y)+x_cont1*scapula.x.dot(thorax.y)+y_cont1*scapula.y.dot(thorax.y)+z_cont1*scapula.z.dot(thorax.y)
z_pos1 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.z)+x_cont1*scapula.x.dot(thorax.z)+y_cont1*scapula.y.dot(thorax.z)+z_cont1*scapula.z.dot(thorax.z)
x_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.x)+x_cont2*scapula.x.dot(thorax.x)+y_cont2*scapula.y.dot(thorax.x)+z_cont2*scapula.z.dot(thorax.x)
y_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.y)+x_cont2*scapula.x.dot(thorax.y)+y_cont2*scapula.y.dot(thorax.y)+z_cont2*scapula.z.dot(thorax.y)
z_pos2 = scapula.masscenter.pos_from(thorax.masscenter).dot(thorax.z)+x_cont2*scapula.x.dot(thorax.z)+y_cont2*scapula.y.dot(thorax.z)+z_cont2*scapula.z.dot(thorax.z)

# Contact forces
f1 = ((x_pos1-mx)/ax)**2+((y_pos1-my)/ay)**2+((z_pos1-mz)/az)**2-1
F1 = 1/2*(f1-sp.sqrt(f1**2+eps**2))
Fx1 = -k_contact*(x_pos1-mx)*(ax**2+ay**2+az**2)/(ax**2)*F1
Fy1 = -k_contact*(y_pos1-my)*(ax**2+ay**2+az**2)/(ay**2)*F1
Fz1 = -k_contact*(z_pos1-mz)*(ax**2+ay**2+az**2)/(az**2)*F1

f2 = ((x_pos2-mx)/ax)**2+((y_pos2-my)/ay)**2+((z_pos2-mz)/az)**2-1
F2 = 1/2*(f2-sp.sqrt(f2**2+eps**2))
Fx2 = -k_contact*(x_pos2-mx)*(ax**2+ay**2+az**2)/(ax**2)*F2
Fy2 = -k_contact*(y_pos2-my)*(ax**2+ay**2+az**2)/(ay**2)*F2
Fz2 = -k_contact*(z_pos2-mz)*(ax**2+ay**2+az**2)/(az**2)*F2

# applying contact forces to contact points in thorax frame
scapula.apply_force(thorax.x*Fx1+thorax.y*Fy1+thorax.z*Fz1,contact_point1)
scapula.apply_force(thorax.x*Fx2+thorax.y*Fy2+thorax.z*Fz2,contact_point2)


In [3]:
# method for solving EoMs
method = me.JointsMethod(thorax,SC,AC,GH)
method.form_eoms()
mm = method.mass_matrix_full
fo = method.forcing_full

In [4]:
# substituting coords (x1..x9) and speeds (x10..x18)
x = sp.symbols('x1:19')
subs_q = {q1:x[0],q2:x[1],q3:x[2],q4:x[3],q5:x[4],q6:x[5],q7:x[6],q8:x[7],q9:x[8]}
subs_u = {u1:x[9],u2:x[10],u3:x[11],u4:x[12],u5:x[13],u6:x[14],u7:x[15],u8:x[16],u9:x[17]}
mm_subs = me.msubs(mm,subs_q,subs_u)
fo_subs = me.msubs(fo,subs_q,subs_u)

In [5]:
text_file = open("yzx_create_eoms.m", "w")
with open("yzx_create_eoms.m", "w") as text_file:
    print('''syms t x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 x12 x13 x14 x15 x16 x17 x18 clavicle_mass scapula_mass humerus_mass c k g rigid2Px rigid2Py rigid2Pz rigid3Cx rigid3Cy rigid3Cz IUxx IUyy IUzz IUxy IUyz IUzx IMxx IMyy IMzz IMxy IMyz IMzx ILxx ILyy ILzz ILxy ILyz ILzx rigid0Px rigid0Py rigid0Pz rigid2Cx rigid2Cy rigid2Cz rigid1Px rigid1Py rigid1Pz rigid1Cx rigid1Cy rigid1Cz x_cont1 y_cont1 z_cont1 x_cont2 y_cont2 z_cont2 k_contact mx my mz ax ay az eps real
q = [x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 x12 x13 x14 x15 x16 x17 x18];
m = [clavicle_mass,scapula_mass,humerus_mass];
IU = [IUxx IUyy IUzz IUxy IUyz IUzx];
IM = [IMxx IMyy IMzz IMxy IMyz IMzx];
IL = [ILxx ILyy ILzz ILxy ILyz ILzx];
rigid0P = [rigid0Px, rigid0Py, rigid0Pz];
rigid1C = [rigid1Cx, rigid1Cy, rigid1Cz];
rigid1P = [rigid1Px, rigid1Py, rigid1Pz];
rigid2C = [rigid2Cx, rigid2Cy, rigid2Cz];
rigid2P = [rigid2Px, rigid2Py, rigid2Pz];
rigid3C = [rigid3Cx, rigid3Cy, rigid3Cz];
cont_params = [mx,my,mz,ax,ay,az,x_cont1,y_cont1,z_cont1,x_cont2,y_cont2,z_cont2,k_contact, eps];

%s
matlabFunction(fo,'file','fo_yzx','vars',{t,q,IU,IM,IL,m,c,k,g,rigid0P,rigid1C,rigid1P,rigid2C,rigid2P,rigid3C,cont_params});
'fo file created'

%s
matlabFunction(mm,'file','mm_yzx','vars',{t,q,IU,IM,IL,m,c,k,g,rigid1C,rigid1P,rigid2C,rigid2P,rigid3C});
'mm file created'

clear all
''' % (sp.octave_code(fo_subs, assign_to = 'fo'),sp.octave_code(mm_subs, assign_to = 'mm')), file=text_file)
    
print("Creating done")

Creating done


In [7]:
## reaction forces in scapula

x1, x2, x3, dx1, dx2, dx3, ddx1, ddx2, ddx3= sp.symbols('x1 x2 x3 dx1 dx2 dx3 ddx1 ddx2 ddx3')
x4, x5, x6, dx4, dx5, dx6, ddx4, ddx5, ddx6= sp.symbols('x4 x5 x6 dx4 dx5 dx6 ddx4 ddx5 ddx6')
x7, x8, x9, dx7, dx8, dx9, ddx7, ddx8, ddx9= sp.symbols('x7 x8 x9 dx7 dx8 dx9 ddx7 ddx8 ddx9')

# reaction forces in scapula frame from accelerations
R_vec = (humerus.masscenter.acc(thorax.frame) + g*thorax.y) * humerus.mass

Rx = R_vec.dot(scapula.x)
Ry = R_vec.dot(scapula.y)
Rz = R_vec.dot(scapula.z)

# q1d = u1 ...
subsq1 = {q1:x1,q2:x2,q3:x3,sp.Derivative(q1, t):dx1,sp.Derivative(q2, t):dx2,sp.Derivative(q3, t):dx3,sp.Derivative(q1, (t, 2)):ddx1,sp.Derivative(q2, (t, 2)):ddx2,sp.Derivative(q3, (t, 2)):ddx3}
subsq2 = {q4:x4,q5:x5,q6:x6,sp.Derivative(q4, t):dx4,sp.Derivative(q5, t):dx5,sp.Derivative(q6, t):dx6,sp.Derivative(q4, (t, 2)):ddx4,sp.Derivative(q5, (t, 2)):ddx5,sp.Derivative(q6, (t, 2)):ddx6}
subsq3 = {q7:x7,q8:x8,q9:x9,sp.Derivative(q7, t):dx7,sp.Derivative(q8, t):dx8,sp.Derivative(q9, t):dx9,sp.Derivative(q7, (t, 2)):ddx7,sp.Derivative(q8, (t, 2)):ddx8,sp.Derivative(q9, (t, 2)):ddx9}
subsu1 = {u1:dx1,u2:dx2,u3:dx3,sp.Derivative(u1, t):ddx1,sp.Derivative(u2, t):ddx2,sp.Derivative(u3, t):ddx3}
subsu2 = {u4:dx4,u5:dx5,u6:dx6,sp.Derivative(u4, t):ddx4,sp.Derivative(u5, t):ddx5,sp.Derivative(u6, t):ddx6}
subsu3 = {u7:dx7,u8:dx8,u9:dx9,sp.Derivative(u7, t):ddx7,sp.Derivative(u8, t):ddx8,sp.Derivative(u9, t):ddx9}

Rx_subs = me.msubs(Rx,subsq1,subsq2,subsq3,subsu1,subsu2,subsu3)
Ry_subs = me.msubs(Ry,subsq1,subsq2,subsq3,subsu1,subsu2,subsu3)
Rz_subs = me.msubs(Rz,subsq1,subsq2,subsq3,subsu1,subsu2,subsu3)
R = sp.Matrix([Rx_subs, Ry_subs, Rz_subs])

text_file = open("create_reaction_from_acc.m", "w")
with open("create_reaction_from_acc.m", "w") as text_file:
    print('''syms t x1 x2 x3 x4 x5 x6 x7 x8 x9 dx1 dx2 dx3 dx4 dx5 dx6 dx7 dx8 dx9 ddx1 ddx2 ddx3 ddx4 ddx5 ddx6 ddx7 ddx8 ddx9
syms rigid2Px rigid2Py rigid2Pz rigid3Cx rigid3Cy rigid3Cz rigid0Px rigid0Py rigid0Pz rigid2Cx rigid2Cy rigid2Cz rigid1Px rigid1Py rigid1Pz rigid1Cx rigid1Cy rigid1Cz real
syms clavicle_mass scapula_mass humerus_mass g
q = [x1 x2 x3 x4 x5 x6 x7 x8 x9];
dq = [dx1 dx2 dx3 dx4 dx5 dx6 dx7 dx8 dx9];
ddq = [ddx1 ddx2 ddx3 ddx4 ddx5 ddx6 ddx7 ddx8 ddx9];
m = [clavicle_mass,scapula_mass,humerus_mass];
rigid0P = [rigid0Px, rigid0Py, rigid0Pz];
rigid1C = [rigid1Cx, rigid1Cy, rigid1Cz];
rigid1P = [rigid1Px, rigid1Py, rigid1Pz];
rigid2C = [rigid2Cx, rigid2Cy, rigid2Cz];
rigid2P = [rigid2Px, rigid2Py, rigid2Pz];
rigid3C = [rigid3Cx, rigid3Cy, rigid3Cz];

%s
matlabFunction(R,'file','GH_react_from_acc','vars',{t,q,dq,ddq,m,g,rigid0P,rigid1C,rigid1P,rigid2C,rigid2P,rigid3C});
'reaction file created'
clear all
''' % sp.octave_code(R,assign_to = 'R'), file=text_file)
    
print("Creating done")

Creating done


(u1(t)*sin(q2(t)) + u3(t))*sin(q2(t)) - (-u1(t)*sin(q3(t))*cos(q2(t)) + u2(t)*cos(q3(t)))*sin(q3(t))*cos(q2(t)) + (u1(t)*cos(q2(t))*cos(q3(t)) + u2(t)*sin(q3(t)))*cos(q2(t))*cos(q3(t))